In [10]:
from __future__ import annotations

from pathlib import Path

import cv2


# --- Configuration ---
# Video file (relative to this notebook's folder)
VIDEO_PATH = Path('files/hero.mp4')

# Output folder for frames
OUTPUT_DIR = Path('frames_hero_webp')

# Image format: 'webp' (recommended), 'jpg', or 'png'
IMAGE_EXT = 'webp'

# Save every Nth frame (1 = every frame)
EVERY_N = 1

# Optional resize (set to None to keep original). Example: (1280, 720)
RESIZE_TO: tuple[int, int] | None = None

# WebP quality (lossy). 0..100 (100 = best quality / largest files)
WEBP_QUALITY = 100

# JPEG quality (only used when IMAGE_EXT == 'jpg') 0..100
JPG_QUALITY = 92

VIDEO_PATH, OUTPUT_DIR

(PosixPath('files/hero.mp4'), PosixPath('frames_hero_webp'))

In [11]:
# Basic validation + metadata
if not VIDEO_PATH.exists():
    raise FileNotFoundError(f'Video not found: {VIDEO_PATH.resolve()}')

cap = cv2.VideoCapture(str(VIDEO_PATH))
if not cap.isOpened():
    raise RuntimeError(f'Failed to open video: {VIDEO_PATH.resolve()}')

fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) or 0)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) or 0)

duration_s = (frame_count / fps) if (fps and frame_count) else None

print('Video:', VIDEO_PATH.resolve())
print('FPS:', fps)
print('Frames:', frame_count)
print('Size:', f'{width}x{height}')
print('Duration (s):', duration_s)

cap.release()

Video: /Users/ghazigharsallah/data/Website/CV/ghazigh.github.io/files/hero.mp4
FPS: 23.23521388716677
Frames: 122
Size: 1920x1080
Duration (s): 5.250651041666667


In [12]:
# Extract frames
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

cap = cv2.VideoCapture(str(VIDEO_PATH))
if not cap.isOpened():
    raise RuntimeError(f'Failed to open video: {VIDEO_PATH.resolve()}')

total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)

# Choose writer params
ext = IMAGE_EXT.lower()
imwrite_params = []
if ext == 'jpg' or ext == 'jpeg':
    imwrite_params = [int(cv2.IMWRITE_JPEG_QUALITY), int(JPG_QUALITY)]
elif ext == 'webp':
    imwrite_params = [int(cv2.IMWRITE_WEBP_QUALITY), int(WEBP_QUALITY)]
elif ext == 'png':
    imwrite_params = []
else:
    cap.release()
    raise ValueError("IMAGE_EXT must be one of: 'webp', 'jpg', 'png'")

saved = 0
read_index = 0

while True:
    ok, frame_bgr = cap.read()
    if not ok:
        break

    # Save every Nth frame
    if (read_index % EVERY_N) != 0:
        read_index += 1
        continue

    if RESIZE_TO is not None:
        frame_bgr = cv2.resize(frame_bgr, RESIZE_TO, interpolation=cv2.INTER_AREA)

    out_name = f'frame_{read_index:06d}.{IMAGE_EXT}'
    out_path = OUTPUT_DIR / out_name

    ok_write = cv2.imwrite(str(out_path), frame_bgr, imwrite_params)
    if not ok_write:
        cap.release()
        raise RuntimeError(f'Failed to write frame to: {out_path.resolve()}')

    saved += 1
    read_index += 1

cap.release()

print(f'Saved {saved} images to: {OUTPUT_DIR.resolve()}')
if total:
    print(f'Read ~{read_index} frames (of {total}) with EVERY_N={EVERY_N}')

Saved 102 images to: /Users/ghazigharsallah/data/Website/CV/ghazigh.github.io/frames_hero_webp
Read ~102 frames (of 122) with EVERY_N=1
